In [1]:

import pandas as pd 
import os
import glob


In [115]:
### Select only Aviation accidents 
os.chdir('../data')
csv_files = glob.glob('*.{}'.format('csv'))
df_append = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index = True)
df_final = df_append.drop_duplicates()

df_final = df_final.loc[df_final['Mode'] == 'Aviation']

### Select only boeing and airbus accidents 
df_final["Make"] = df_final["Make"].fillna("")
df_boeing_airbus = df_final[df_final.apply(lambda x: x['Make'].upper(),axis= 1).str.contains('|'.join(['BOEING','AIRBUS']))]



In [116]:
### Create separate columns for datetime
df_boeing_airbus['UTC_YEAR'] = pd.to_datetime(df_boeing_airbus['EventDate']).dt.year
df_boeing_airbus['UTC_MTH'] = pd.to_datetime(df_boeing_airbus['EventDate']).dt.month
df_boeing_airbus['UTC_DAY'] = pd.to_datetime(df_boeing_airbus['EventDate']).dt.day
df_boeing_airbus['UTC_HOUR'] = pd.to_datetime(df_boeing_airbus['EventDate']).dt.hour
df_boeing_airbus['UTC_WEEKDAY'] = pd.to_datetime(df_boeing_airbus['EventDate']).dt.day_name()

/var/folders/49/75nljxy5475cbm9jg654pl1w0000gn/T/ipykernel_15287/81193258.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_boeing_airbus['UTC_YEAR'] = pd.to_datetime(df_boeing_airbus['EventDate']).dt.year
/var/folders/49/75nljxy5475cbm9jg654pl1w0000gn/T/ipykernel_15287/81193258.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_boeing_airbus['UTC_MTH'] = pd.to_datetime(df_boeing_airbus['EventDate']).dt.month
/var/folders/49/75nljxy5475cbm9jg654pl1w0000gn/T/ipykernel_15287/81193258.py:4: SettingWit

In [117]:
### The number of accidents per year 
df_boeing_airbus['UTC_YEAR'].value_counts().reset_index().to_csv('year_count.csv')

In [118]:
### Transfer collision incidents involving both airbus and boeing to separate rows 

mask = df_boeing_airbus['Make'].isin(["Boeing, Airbus","AIRBUS, BOEING","AIRBUS INDUSTRIE, BOEING","BOEING, AIRBUS"])
df_duplicate = df_boeing_airbus[df_boeing_airbus['Make'].isin(["Boeing, Airbus","AIRBUS, BOEING","AIRBUS INDUSTRIE, BOEING","BOEING, AIRBUS"])]
df_unique  = df_boeing_airbus[~df_boeing_airbus['Make'].isin(["Boeing, Airbus","AIRBUS, BOEING","AIRBUS INDUSTRIE, BOEING","BOEING, AIRBUS"])]

df_temp = pd.concat([df_duplicate, df_duplicate[mask]]).sort_index().reset_index(drop=True)
df_temp['Make_final'] = ['Boeing'] * 24 +  ['Airbus'] * 24 


df_unique['Make_final'] = df_unique['Make'].apply(lambda x: 'Boeing' if 'BOEING' in x.upper() else 'Airbus')

df_boeing_airbus_final = pd.concat([df_unique,df_temp])

/var/folders/49/75nljxy5475cbm9jg654pl1w0000gn/T/ipykernel_15287/219017469.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp = pd.concat([df_duplicate, df_duplicate[mask]]).sort_index().reset_index(drop=True)
/var/folders/49/75nljxy5475cbm9jg654pl1w0000gn/T/ipykernel_15287/219017469.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['Make_final'] = df_unique['Make'].apply(lambda x: 'Boeing' if 'BOEING' in x.upper() else 'Airbus')


In [126]:
### The number incidents per year by make 
df_boeing_airbus_final.groupby(['Make_final'])['UTC_YEAR'].value_counts().reset_index().to_csv('year_make_count.csv')

In [128]:
df_boeing_airbus_final[['UTC_YEAR','FatalInjuryCount', 'SeriousInjuryCount',
       'MinorInjuryCount']].groupby(['UTC_YEAR'])['FatalInjuryCount'].sum()


UTC_YEAR
2010    517.0
2011     81.0
2012    280.0
2013     76.0
2014    707.0
2015    389.0
2016    134.0
2017     39.0
2018    308.0
2019    175.0
2020    276.0
2021     70.0
2022    145.0
2023      2.0
2024      6.0
Name: FatalInjuryCount, dtype: float64